###Import and load datasets

In [ ]:
#! pip install python-terrier

!pip install --upgrade python-terrier

!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 95 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
     |████████████████████████████████| 69 kB 6.3 MB/s 
     |████████████████████████████████| 255 kB 56.7 MB/s 
     |████████████████████████████████| 41 kB 271 kB/s 
     |████████████████████████████████| 6.4 MB 43.5 MB/s 
     |████████████████████████████████| 1.8 MB 53.8 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 294 kB 60.6 MB/s 
     |████████████████████████████████| 596 kB 35.7 MB/s 
     |████████████████████████████████| 126 kB 52.7 MB/s 
     |████████████████████████████████| 291 kB 48.4 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.2-py3-none-any.whl size=101953 sha256=512b24d822be4fac7b5fe4dd6da4f7e5056567faa07caf61ef52915c4555cbb3
  Stored in directory: /root/.cac

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#libraries

import pyterrier as pt
#from pyterrier import IndexingType
import pandas as pd
import regex as re
import string 
from nltk.stem.porter import *
from nltk.tokenize import WordPunctTokenizer
import pandas as pd


In [ ]:
#load datasets and init pyterrier

if not pt.started():
  pt.init()

root_dir = '/content/gdrive/MyDrive'
base_dir = root_dir + '/Progetto IR/data'
collection = pd.read_pickle(dir + "/collection.pkl")
qrels = pd.read_pickle(dir + "/qrels.pkl")
queries = pd.read_pickle(dir + "/queries.pkl")


NameError: ignored

###Pre-processing 

In [ ]:
covid19_synonyms_nospace = ['covid19',
                    'covid2019',
                    'coronavirusdisease19',
                    'coronavirusdisease2019',
                    'coviddisease2019',
                    'coviddisease19',
                    'sarscov2', 
                    '2019ncov',
                    'ncov2019',
                    'coronavirus2019',
                    'coronavirus2',
                    'wuhanpneumonia',
                    'wuhanvirus',
                    'wuhancoronavirus',
                    'novelcoronavirus',
                    'novelcovid'
                    ]
covid19_synonyms_space =[
                         'covid 19',
                         'covid 2019',
                         'coronavirus disease 19',
                         'ncov 2019',
                         'n cov2019',
                         'n cov 2019',
                         '2019n cov',
                         '2019 ncov',
                         '2019 n cov',
                         'coronavirus 2019',
                         'coronavirus 2',
                         'wuhan pneumonia',
                         'wuhan virus',
                         'wuhan coronavirus',
                         'novel coronavirus',
                         'novel covid',
                         'covid disease 2019',
                         'coronavirus disease 2019',
                         'covid disease 19',
                         'sars cov 2',
                         'sars cov2',
                         'sarscov 2'
]

In [ ]:

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation+'‐'+'′'+'–'+'‘'+'’'+'“'+'”'])
    return punctuationfree

import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)

  tokenized_text_without_stopwords = []
  for token in tokenized_text:
    if token.lower() not in stop_words:
      tokenized_text_without_stopwords.append(token)
  return " ".join(tokenized_text_without_stopwords)

def tokenization(text):
  tokenized_text = WordPunctTokenizer().tokenize(text)
  return tokenized_text

def covid_preprocess(text):
  for s in covid19_synonyms_space:
    text = text.replace(s, s.replace(" ", ""))
  return text

def remove_number_after_space(text):
  text = re.sub('\s(\d)+', ' ', text)
  return text


def porter_stemmer(text):
  stemmer = PorterStemmer()
  return_text = stemmer.stem(text)

  return return_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing_first_part(t):
  t = t.apply(lambda x: x.lower())
  t = t.apply(lambda x: re.sub('-', ' ', x))
  t = t.apply(lambda x: remove_punctuation(x))
  t = t.apply(lambda x: remove_stopwords(x))
  #covid pre
  t = t.apply(lambda x: covid_preprocess(x))
  t = t.apply(lambda x: remove_number_after_space(x))
  return t

In [ ]:
display(queries.head())

queries['adhoc_preprocess'] = preprocessing_first_part(queries['query_adhoc'])
queries['desc_preprocess'] = preprocessing_first_part(queries['query_desc'])
queries['nar_preprocess'] = preprocessing_first_part(queries['query_nar'])

queries.head()


,index,qid,query,query_adhoc,query_desc,query_nar
0,0,1,NaN,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,2,3,NaN,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,3,4,NaN,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,4,5,NaN,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


,index,qid,query,query_adhoc,query_desc,query_nar,adhoc_preprocess,desc_preprocess,nar_preprocess
0,0,1,NaN,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...,coronavirus origin,origin covid19,seeking range information sarscov2 viruss orig...
1,1,2,NaN,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...,coronavirus response weather changes,coronavirus respond changes weather,seeking range information sarscov2 virus viabi...
2,2,3,NaN,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...,coronavirus immunity,sarscov2 infected people develop immunity cros...,seeking studies immunity developed due infecti...
3,3,4,NaN,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...,people die coronavirus,causes death covid19,studies looking mechanisms death covid19
4,4,5,NaN,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...,animal models covid19,drugs active sars cov sarscov2 animal studies,papers describe results testing drugs bind spi...


In [ ]:
queries_adhoc = queries[['qid', 'adhoc_preprocess']]
queries_adhoc.rename(columns = {"adhoc_preprocess" : "query"}, inplace=True)

queries_desc = queries[['qid', 'desc_preprocess']]
queries_desc.rename(columns = {"desc_preprocess" : "query"}, inplace=True)

queries_narrative = queries[['qid', 'nar_preprocess']]
queries_narrative.rename(columns = {"nar_preprocess" : "query"}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


###Indexing

In [ ]:
pt.set_property("termpipelines", "")
pt.set_property("termpipelines", "Stopwords")
pt.set_property("termpipelines", "PorterStemmer")

pd_indexer = pt.DFIndexer("/pd_index", overwrite = True)
indexref = pd_indexer.index(collection["text"], collection["docno"], collection["title"])


14:31:57.431 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
14:34:23.391 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 54886 empty documents
14:34:23.434 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [ ]:
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())
print(index.getLexicon()["virus"].toString())

Number of documents: 192509
Number of terms: 149557
Number of postings: 15053000
Number of fields: 0
Number of tokens: 26884365
Field names: []
Positions:   false

term230 Nt=14501 TF=29478 maxTF=2147483647 @{0 18125399 7}


###Documents retrive and re-ranking by date

In [ ]:
from pyterrier.measures import *
import onir_pt

tfidf = pt.BatchRetrieve(index, wmodel="DPH")
res = tfidf.transform(queries_adhoc[queries_adhoc.qid=='19'])
display(res.head(15))


,qid,docid,docno,rank,score,query
0,19,97351,i0ll585x,0,17.255356,alcohol sanitizer kills coronavirus
1,19,166033,d26y5291,1,17.255356,alcohol sanitizer kills coronavirus
2,19,130032,uhhk4t7f,2,15.707038,alcohol sanitizer kills coronavirus
3,19,130033,9iyyqqmm,3,15.707038,alcohol sanitizer kills coronavirus
4,19,130034,rpre7b8w,4,15.707038,alcohol sanitizer kills coronavirus
5,19,162652,20ipkh78,5,15.698795,alcohol sanitizer kills coronavirus
6,19,132368,y777xosr,6,15.588764,alcohol sanitizer kills coronavirus
7,19,155496,wr404h18,7,15.588764,alcohol sanitizer kills coronavirus
8,19,117342,eevs62xf,8,15.173217,alcohol sanitizer kills coronavirus
9,19,162965,hma2kvn2,9,15.165827,alcohol sanitizer kills coronavirus


In [ ]:
def re_ranking_by_date(df_res):
  res_date = pd.merge(df_res, collection[['docno','date']], on=['docno'])
  res_date['date'] = pd.to_datetime(res_date.date)
  res_date['score_int'] = res_date.score.apply(lambda x: int(x))
  res_date = res_date.sort_values(by=['score_int','date'], ascending=False)
  res_date['rank_date'] = range(0, len(res_date))
  return res_date

In [ ]:
after_re_rank_date = re_ranking_by_date(res)

In [ ]:
after_re_rank_date

,qid,docid,docno,rank,score,query,date,score_int,rank_date
1,19,166033,d26y5291,1,17.255356,alcohol sanitizer kills coronavirus,2020-05-11,17,0
0,19,97351,i0ll585x,0,17.255356,alcohol sanitizer kills coronavirus,2020-01-01,17,1
9,19,162965,hma2kvn2,9,15.165827,alcohol sanitizer kills coronavirus,2020-06-27,15,2
7,19,155496,wr404h18,7,15.588764,alcohol sanitizer kills coronavirus,2020-06-18,15,3
5,19,162652,20ipkh78,5,15.698795,alcohol sanitizer kills coronavirus,2020-04-20,15,4
...,...,...,...,...,...,...,...,...,...
1013,19,83332,2fxupg5j,999,4.386407,alcohol sanitizer kills coronavirus,1987-01-01,4,1009
880,19,159155,f5yoliqm,868,4.576155,alcohol sanitizer kills coronavirus,1986-08-31,4,1010
946,19,168049,1rzbaj02,932,4.478032,alcohol sanitizer kills coronavirus,1983-06-30,4,1011
783,19,133900,id4lozjr,771,4.845237,alcohol sanitizer kills coronavirus,1981-05-31,4,1012


In [ ]:
def re_ranking_all_by_date(df_res, l):
  after = []
  for i in list(l['qid']):
    df_q = df_res[df_res.qid == i]
    after.append(re_ranking_by_date(df_q))
  return pd.concat(after)


In [ ]:
res_all = tfidf.transform(queries_adhoc)
display(res_all.head(5))

,qid,docid,docno,rank,score,query
0,1,175892,zy8qjaai,0,8.485659,coronavirus origin
1,1,82224,8ccl9aui,1,8.177735,coronavirus origin
2,1,84953,ax6v6ham,2,7.864674,coronavirus origin
3,1,93674,r0peje13,3,7.847122,coronavirus origin
4,1,140409,llv3cvdr,4,7.847122,coronavirus origin


In [ ]:
after_re_rank_date_all = re_ranking_all_by_date(res_all, queries_adhoc)

In [ ]:
display(after_re_rank_date_all)

,qid,docid,docno,rank,score,query,date,score_int,rank_date
0,1,175892,zy8qjaai,0,8.485659,coronavirus origin,2018-07-28,8,0
1,1,82224,8ccl9aui,1,8.177735,coronavirus origin,2004-01-01,8,1
13,1,68581,4dtk1kyh,13,7.465062,coronavirus origin,2020-07-01,7,2
20,1,71023,qw05apnf,20,7.297666,coronavirus origin,2020-06-12,7,3
24,1,32725,1pnc889f,24,7.252321,coronavirus origin,2020-05-21,7,4
...,...,...,...,...,...,...,...,...,...
808,50,83065,wxqyqmtx,805,5.659729,mrna vaccine coronavirus,1984-01-01,5,998
731,50,176473,39n0713g,728,5.751832,mrna vaccine coronavirus,1983-02-11,5,999
665,50,89011,dufre0ri,662,5.848159,mrna vaccine coronavirus,1983-01-01,5,1000
653,50,84323,een09dzg,650,5.861207,mrna vaccine coronavirus,1977-01-01,5,1001
